In [1]:
import pandas as pd

import sys
import pathlib

import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox, Dropdown, IntSlider, interact
from IPython.display import display

%matplotlib inline

In [2]:
import logging
logger = logging.basicConfig(level=logging.DEBUG)

In [3]:
#sys.path.append("G:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/MicroMet")
#sys.path.append("C:/Users/kladig/Documents/GitHub/MicroMet")
sys.path.append("../../micromet")
import micromet

DEBUG:rasterio.session:Could not import boto3, continuing with reduced functionality.


In [4]:
# dictionary of station ids as keys and values as subdirectories
site_folders = {'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_Mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTG':'Green River',
                }

compdf = {}
am = micromet.AmerifluxDataProcessor()

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('G:/Shared drives/UGS_Flux/Data_Downloads/')
    amflux_out = 'G:/Shared drives/UGS_Flux/Data_Processing/Ameriflux/'
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(raw_data, config_path="../micromet/reformatter_vars.yml", logger=logger)
        am_df = am_data.et_data
        compdf[key] = am_df
        micromet.outfile(am_df, key, amflux_out)

        
cdf = pd.concat(compdf,axis=0)

# make lists for selection dropdowns in the interactive parts of the code
categories = cdf.index.get_level_values(0).unique()
fields = list(cdf.columns)

US-UTD


US-UTB


US-UTJ


US-UTW


US-UTE


...

US-UTP


c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning:

Degrees of freedom <= 0 for slice.

c:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\Notebooks\../../micromet\micromet\converter.py:1012: RuntimeWarning:

Mean of empty slice

c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning:

Degrees of freedom <= 0 for slice.

c:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\Notebooks\../../micromet\micromet\converter.py:1012: RuntimeWarning:

Mean of empty slice

c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning:

Degrees of freedom <= 0 for slice.

c:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\Notebooks\../../micromet\micromet\converter.py:1012: RuntimeWarning:

Mean of empty slice

c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning:

De

US-CdM


US-UTV


US-UTG


In [ ]:

def filter_data(index_value):
    #index_value = change['new']

    # Check if input index exists in DataFrame
    if index_value in cdf.index:
        display(cdf.loc[index_value])
    else:
        print(f"Index {index_value} does not exist in DataFrame.")

index_input = interact(filter_data,
    index_value = widgets.Dropdown(
        options=categories,
        value=categories[0],
        description='Categories',
        disabled=False
    ))

# Interactive intital Plots

In [ ]:
# Create interactive widgets with date range
interact(
    micromet.plot_timeseries_daterange,
    input_df = widgets.fixed(cdf),
    selected_station=widgets.Dropdown(
        options=categories,
        value=categories[0],
        description='Categories',
        disabled=False
    ),
    selected_field=widgets.Dropdown(
        options=fields,
        value=fields[0],
        description='Field',
        disabled=False
    ),
    start_date=widgets.DatePicker(
        description='Start Date',
        value=pd.to_datetime('2020-01-01')
    ),
    end_date=widgets.DatePicker(
        description='End Date',
        value=pd.to_datetime('2024-12-31')
    )
)

save_button = widgets.Button(description="Save Plot")
save_button.on_click(micromet.save_plot)
display(save_button)


In [ ]:
UTP = cdf.loc['US-UTP']

In [ ]:
UTP_filtered = UTP[(UTP.index >= pd.to_datetime('2024-06-07 09:30'))] #202310210000

In [ ]:
UTP_filtered

In [ ]:
UTP_filtered.to_csv(amflux_out + 'US-UTP/US-UTP_HH_202406070930_202502101030.csv')

In [ ]:
UTJ = cdf.loc['US-UTJ']

In [ ]:
UTJ_filtered = UTJ.drop(columns=['TA_1_4_1'])

In [ ]:
max_col1 = UTJ['P'].max()
print(max_col1)

In [ ]:
UTJ_filtered['P'] = np.nan

In [ ]:
UTJ_filtered = UTJ_filtered[(UTJ_filtered.index >= pd.to_datetime('2024-06-26 12:30'))] #202310210000

In [ ]:
UTJ_filtered

In [ ]:
UTJ_filtered.to_csv(amflux_out + 'US-UTJ/US-UTJ_HH_202406261230_202502191400.csv')

In [ ]:
UTV = cdf.loc['US-UTV']

In [ ]:
UTV

In [ ]:
UTV_filtered = UTV

In [ ]:
# Define the date boundaries
start_date = pd.to_datetime('2023-10-15')
end_date = pd.to_datetime('2023-11-21')

# Create a boolean mask to select rows within August 2024
mask = (UTV_filtered.index >= start_date) & (UTV_filtered.index <= end_date)

# Set the values of 'tw' to -9999 within August 2024
UTV_filtered.loc[mask, 'TA_1_4_1'] = -9999
UTV_filtered.loc[mask, 'TS_1_2_1'] = -9999
UTV_filtered.loc[mask, 'TS_1_1_1'] = -9999
UTV_filtered.loc[mask, 'NETRAD'] = -9999

In [ ]:
# Define the date boundaries
start_date2 = pd.to_datetime('2024-02-20')
end_date2 = pd.to_datetime('2024-03-14')

# Create a boolean mask to select rows within August 2024
mask = (UTV_filtered.index >= start_date2) & (UTV_filtered.index <= end_date2)

# Set the values of 'tw' to -9999 within August 2024
UTV_filtered.loc[mask, 'TS_1_2_1'] = -9999
UTV_filtered.loc[mask, 'SW_IN'] = -9999

In [ ]:
# Define the date boundaries
start_date3 = pd.to_datetime('2024-04-18')
end_date3 = pd.to_datetime('2024-10-26')

# Create a boolean mask to select rows within August 2024
mask = (UTV_filtered.index >= start_date3) & (UTV_filtered.index <= end_date3)

# Set the values of 'tw' to -9999 within August 2024
UTV_filtered.loc[mask, 'TA_1_4_1'] = -9999
UTV_filtered.loc[mask, 'LW_OUT'] = -9999
UTV_filtered.loc[mask, 'LW_IN'] = -9999
UTV_filtered.loc[mask, 'SW_OUT'] = -9999

In [ ]:
def replace_low_values(df, column_name, threshold=150, replacement=-9999):
  """
  Replaces values in a specified column of a Pandas DataFrame that are less than a threshold with a replacement value.

  Args:
    df: Pandas DataFrame.
    column_name: Name of the column to modify.
    threshold: Threshold value. Values less than this will be replaced.
    replacement: Value to replace values below the threshold.

  Returns:
    Pandas DataFrame with the modified column.
  """

  df[column_name] = np.where(df[column_name] < threshold, replacement, df[column_name])
  return df


In [ ]:
UTV_filtered = replace_low_values(UTV_filtered, 'LW_IN')

In [ ]:
UTV_filtered = replace_low_values(UTV_filtered, 'LW_OUT')

In [ ]:
UTV_filtered

In [ ]:
UTV_filtered.to_csv(amflux_out + 'US-UTV/US-UTV_HH_202309271530_202412111330.csv')

In [ ]:
UTE = cdf.loc['US-UTE']

In [ ]:
UTE

In [ ]:
UTD = cdf.loc['US-UTD']

In [ ]:
UTD

In [ ]:
CdM = cdf.loc['US-CdM']

In [ ]:
CdM

In [ ]:
CdM_filtered = CdM[(CdM.index >= pd.to_datetime('2023-10-21 00:00'))] #202310210000

In [ ]:
CdM_filtered

In [ ]:
def add_hour_after_date(df, datetime_col, cutoff_date='202312120000'):
  """
  Subtracts one hour from a datetime column for every date after a specified cutoff date.

  Args:
    df: The pandas DataFrame containing the datetime column.
    datetime_col: The name of the datetime column.
    cutoff_date: The cutoff date (default: '2024-05-01').

  Returns:
    A new DataFrame with the modified datetime column.
  """

  cutoff_date = pd.to_datetime(cutoff_date, format = '%Y%m%d%H%M')
  df.loc[df[datetime_col] > cutoff_date, datetime_col] += pd.to_timedelta('1 hour') 

  return df

  df.drop('date_only', axis=1, inplace=True) 

  return df

In [ ]:
def drop_two_zeros_from_column(df, column_name):
  """
  Drops the last two zeros from the end of every string in a specified column of a pandas DataFrame.

  Args:
    df: The pandas DataFrame.
    column_name: The name of the column containing the strings.

  Returns:
    A new DataFrame with the modified column.
  """

  def drop_two_trailing_zeros(string):
    if string.endswith("00"):
      return string[:-2]
    else:
      return string

  df[column_name] = df[column_name].apply(drop_two_trailing_zeros)
  return df

In [ ]:
CdM_filtered['TIMESTAMP_START'] = pd.to_datetime(CdM_filtered['TIMESTAMP_START'], format = '%Y%m%d%H%M')
CdM_filtered['TIMESTAMP_END'] = pd.to_datetime(CdM_filtered['TIMESTAMP_END'], format = '%Y%m%d%H%M')

CdM_filtered = add_hour_after_date(CdM_filtered, 'TIMESTAMP_START')
CdM_filtered = add_hour_after_date(CdM_filtered, 'TIMESTAMP_END')

In [ ]:
CdM_filtered['TIMESTAMP_START'] = CdM_filtered['TIMESTAMP_START'].astype('str').str.replace('-', '').str.replace(' ', '').str.replace(':', '')
CdM_filtered['TIMESTAMP_END'] = CdM_filtered['TIMESTAMP_END'].astype('str').str.replace('-', '').str.replace(' ', '').str.replace(':', '')

In [ ]:
CdM_filtered = drop_two_zeros_from_column(CdM_filtered, 'TIMESTAMP_START')

CdM_filtered = drop_two_zeros_from_column(CdM_filtered, 'TIMESTAMP_END')

In [ ]:
CdM_filtered

In [ ]:
CdM_filtered.to_csv(amflux_out + 'US-CdM/US-CdM_HH_202310210000_202502191730.csv')

In [ ]:
UTM = cdf.loc['US-UTM']

In [ ]:
#start_date = pd.to_datetime('202210261430')
#to_delete = [pd.to_datetime('2022-10-26 14:30'), pd.to_datetime('2024-10-09 06:00')]
#UTM = UTM[~(UTM.index.strftime('%Y-%m-%d %H:%M').isin(to_delete))]
#to_save = [pd.to_datetime('2022-10-26 14:30'), pd.to_datetime('2024-10-09 06:00')] 
#UTM2 = UTM[~UTM.index.isin(to_save)] 
UTM_filtered = UTM[(UTM.index >= pd.to_datetime('2022-10-26 15:00'))] 


In [ ]:
UTM_filtered

In [ ]:
UTM_filtered = UTM_filtered.drop('TA_1_4_1', axis = 1)

In [ ]:
UTM_filtered = UTM_filtered.drop('RH_1_3_1', axis = 1)

In [ ]:
# Define the date boundaries
start_date = pd.to_datetime('2023-05-01')
end_date = pd.to_datetime('2024-05-01')

# Create boolean masks for the date ranges
before_05_2023 = UTM_filtered.index < start_date
after_05_2024 = UTM_filtered.index > end_date

# Set 'TA' to -9999 for the specified date ranges
UTM_filtered.loc[before_05_2023, 'TA_1_3_1'] = -9999
UTM_filtered.loc[after_05_2024, 'TA_1_3_1'] = -9999
UTM_filtered.loc[before_05_2023, 'RH_1_3_1'] = -9999

In [ ]:
start_date2 = pd.to_datetime('2024-08-01')
end_date2 = pd.to_datetime('2024-08-31')

# Create a boolean mask to select rows within August 2024
mask = (UTM_filtered.index >= start_date2) & (UTM_filtered.index <= end_date2)

# Set the values of 'tw' to -9999 within August 2024
UTM_filtered.loc[mask, 'LW_IN'] = -9999


In [ ]:
start_date3 = pd.to_datetime('2024-10-10')
end_date3 = pd.to_datetime('2024-10-18')

# Create a boolean mask to select rows within August 2024
mask = (UTM_filtered.index >= start_date3) & (UTM_filtered.index <= end_date3)

# Set the values of 'tw' to -9999 within August 2024
UTM_filtered.loc[mask, 'LW_IN'] = -9999

In [ ]:
UTM_filtered.reset_index(drop=True, inplace=True)

In [ ]:
UTM_filtered

In [ ]:
def subtract_hour_after_date(df, datetime_col, cutoff_date='202405010000'):
  """
  Subtracts one hour from a datetime column for every date after a specified cutoff date.

  Args:
    df: The pandas DataFrame containing the datetime column.
    datetime_col: The name of the datetime column.
    cutoff_date: The cutoff date (default: '2024-05-01').

  Returns:
    A new DataFrame with the modified datetime column.
  """

  cutoff_date = pd.to_datetime(cutoff_date, format = '%Y%m%d%H%M')
  df.loc[df[datetime_col] > cutoff_date, datetime_col] -= pd.to_timedelta('1 hour') 

  return df

  df.drop('date_only', axis=1, inplace=True) 

  return df

In [ ]:
dtypelist = UTM_filtered.dtypes
print(dtypelist)

In [ ]:
UTM_filtered['TIMESTAMP_START'] = pd.to_datetime(UTM_filtered['TIMESTAMP_START'], format = '%Y%m%d%H%M')

In [ ]:
UTM_filtered['TIMESTAMP_END'] = pd.to_datetime(UTM_filtered['TIMESTAMP_END'], format = '%Y%m%d%H%M')

In [ ]:
UTM_filtered = subtract_hour_after_date(UTM_filtered, 'TIMESTAMP_END', '202405010000')

In [ ]:
UTM_filtered = subtract_hour_after_date(UTM_filtered, 'TIMESTAMP_START', '202405010000')

In [ ]:
UTM_filtered['TIMESTAMP_START'] = UTM_filtered['TIMESTAMP_START'].astype('str').str.replace('-', '').str.replace(' ', '').str.replace(':', '')
UTM_filtered['TIMESTAMP_END'] = UTM_filtered['TIMESTAMP_END'].astype('str').str.replace('-', '').str.replace(' ', '').str.replace(':', '')

In [ ]:
UTM_filtered

In [ ]:
def drop_two_zeros_from_column(df, column_name):
  """
  Drops the last two zeros from the end of every string in a specified column of a pandas DataFrame.

  Args:
    df: The pandas DataFrame.
    column_name: The name of the column containing the strings.

  Returns:
    A new DataFrame with the modified column.
  """

  def drop_two_trailing_zeros(string):
    if string.endswith("00"):
      return string[:-2]
    else:
      return string

  df[column_name] = df[column_name].apply(drop_two_trailing_zeros)
  return df

In [ ]:
UTM_filtered = drop_two_zeros_from_column(UTM_filtered, 'TIMESTAMP_START')

UTM_filtered = drop_two_zeros_from_column(UTM_filtered, 'TIMESTAMP_END')

In [ ]:
UTM_filtered

In [ ]:
UTM_filtered.to_csv(amflux_out + 'US-UTM/US-UTM_HH_202210261500_202412110700.csv')

In [ ]:
UTW = cdf.loc['US-UTW']

In [ ]:
UTW_filtered = UTW

In [ ]:
UTW_filtered

In [ ]:
start_date4 = pd.to_datetime('2024-10-12')
end_date4 = pd.to_datetime('2024-10-18')

# Create a boolean mask to select rows within August 2024
mask = (UTW_filtered.index >= start_date4) & (UTW_filtered.index <= end_date4)

# Set the values of 'tw' to -9999 within August 2024
UTW_filtered.loc[mask, 'LW_IN'] = -9999
UTW_filtered.loc[mask, 'LW_OUT'] = -9999
UTW_filtered.loc[mask, 'SW_IN'] = -9999

In [ ]:
UTW_filtered

In [ ]:
start_date5 = pd.to_datetime('2022-09-22')
end_date5 = pd.to_datetime('2022-10-22')

# Create a boolean mask to select rows within August 2024
mask = (UTW_filtered.index >= start_date5) & (UTW_filtered.index <= end_date5)

# Set the values of 'tw' to -9999 within August 2024
UTW_filtered.loc[mask, 'T_SONIC'] = -9999

In [ ]:
UTW_filtered

In [ ]:
start_date6 = pd.to_datetime('2023-02-09')
end_date6 = pd.to_datetime('2023-02-17')

# Create a boolean mask to select rows within August 2024
mask = (UTW_filtered.index >= start_date6) & (UTW_filtered.index <= end_date6)

# Set the values of 'tw' to -9999 within August 2024
UTW_filtered.loc[mask, 'T_SONIC'] = -9999

In [ ]:
UTW_filtered

In [ ]:
start_date7 = pd.to_datetime('2022-09-22')
end_date7 = pd.to_datetime('2022-10-22')

# Create a boolean mask to select rows within August 2024
mask = (UTW_filtered.index >= start_date7) & (UTW_filtered.index <= end_date7)

# Set the values of 'tw' to -9999 within August 2024
UTW_filtered.loc[mask, 'T_SONIC'] = -9999

In [ ]:
UTW_filtered = UTW_filtered.drop('P', axis = 1)

In [ ]:
UTW_filtered

In [ ]:
UTW_filtered.to_csv(amflux_out + 'US-UTW/US-UTW_HH_202105200830_202412111300.csv')

In [ ]:
raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/Wellington')
value = 'm20240723'
raw_met_data = micromet.raw_file_compile(raw_fold, value, search_str = "*AmeriFlux*.dat")

print(pd.to_datetime(raw_met_data['TIMESTAMP_START'][0],format='%Y%m%d%H%M'))

raw_met_data['datetime'] = pd.date_range(start='2024-07-11 06:00:00', freq='30min', periods=len(raw_met_data))
raw_met_data.set_index('datetime', inplace=True)
#micromet.Reformatter(raw_met_data)

In [ ]:
data.rename(columns={'WD':'csat_WD'})['csat_WD']

In [ ]:
df_wd = pd.concat([data.rename(columns={'WD':'csat_WD','WS':'csat_WS'}),
                  raw_met_data.rename(columns={'WD':'young_WD','WS':'young_WS'}).shift(-1)], axis=1)
df_wd = df_wd[['csat_WD','young_WD','csat_WS','young_WS']].dropna(axis=1, how='all')
df_wd['csat_wd_rad'] = pg.circ_axial(pg.convert_angles(df_wd['csat_WD'],positive=True),2)
csat_wd_mean = pg.circ_mean(pg.convert_angles(df_wd['csat_WD']))
df_wd['young_wd_rad'] = pg.circ_axial(pg.convert_angles(df_wd['young_WD'],positive=True),2)
young_wd_mean = pg.circ_mean(pg.convert_angles(df_wd['young_WD']))



print(csat_wd_mean, young_wd_mean)
pg.circ_corrcc(df_wd['csat_wd_rad'], df_wd['young_wd_rad'], correction_uniform=True)

In [ ]:
df_wd.loc[(df_wd['young_wd_rad']>3.2) & (df_wd['csat_wd_rad']<2),'csat_wd_rad'] = df_wd.loc[(df_wd['young_wd_rad']<3.2) & (df_wd['csat_wd_rad']>2),'csat_wd_rad'] + 6.25

fig = px.scatter(df_wd, x='csat_wd_rad', y='young_wd_rad', trendline="ols",color=df_wd.index.hour)
fig.show()

In [ ]:

#fig = px.line(df_wd, y=["young_WS","csat_WS"],
#              title='custom tick labels')
fig = px.scatter(df_wd, x='young_WS', y='csat_WS', trendline="ols",color=df_wd.index.hour)
fig.show()

In [ ]:


fig = px.line(df_wd, y=["young_WD","csat_WD"],
              title='custom tick labels')

fig.show()

In [ ]:
#raw_met_data['SW_IN'].plot()

comp_col = 'LW_OUT'

sql = "SELECT * FROM groundwater.amfluxeddy WHERE (groundwater.amfluxeddy.stationid = 'US-UTW') AND (groundwater.amfluxeddy.timestamp_start >= 202407110000)"
data = pd.read_sql(sql, con=engine)
data = data.replace(-9999, np.nan)
data.set_index('datetime_start', inplace=True)
for col in data.columns:
    data.rename(columns={col:col.upper()}, inplace=True)
#data['SW_IN'].plot()

df = pd.concat([data.rename(columns={f'{comp_col}':f'{comp_col}_HF'})[f'{comp_col}_HF'].shift(-1),
                raw_met_data.rename(columns={f'{comp_col}':f'{comp_col}_AP'})[f'{comp_col}_AP']],axis=1).dropna()


y = df[f'{comp_col}_HF']
x = df[f'{comp_col}_AP']
X = sm.add_constant(x)
model = sm.OLS(y,X)
results = model.fit()
print(results.params)



fig = px.scatter(df, x=f'{comp_col}_HF', y=f'{comp_col}_AP', trendline="ols",color=df.index.hour)

ax = pg.plot_blandaltman(x, y)

fig.show()
print(results.summary())

In [ ]:
d = pg.plot_blandaltman(x, y)

In [ ]:
import plotly.graph_objects as go

dt_tm = pd.to_datetime('2024-07-15 11:00')
lwin  = data.loc[dt_tm,'LW_IN']
lwout = data.loc[dt_tm,'LW_OUT']
swin = data.loc[dt_tm,'SW_IN']
swout = data.loc[dt_tm,'SW_OUT']
shf = data.loc[dt_tm,'G']
le = data.loc[dt_tm,'LE']
h = data.loc[dt_tm,'H']
netrad = data.loc[dt_tm,'NETRAD']


import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = ["Longwave In", "Shortwave In","Energy In", "Net Radiation","Longwave Out", "Shortwave Out","Soil Heat Flux","Latent Energy","Sensible Heat"],
        color = ['red','yellow','orange','red','yellow','green','blue','orange']
    ),
    link = dict(
        source = [0,1,2,2,2,3,3,3,], # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = [2,2,3,4,5,6,7,8,],
        value = [lwin,swin,netrad,lwout,swout,shf,le,h],
        color = ['red','yellow','red','yellow','green','blue','orange']
    ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
data.loc[dt_tm,'ALB']

In [ ]:
print(df.ptests(paired=True, stars=False))

In [ ]:
sql = "SELECT * FROM groundwater.amfluxeddy WHERE groundwater.amfluxeddy.stationid = 'US-UTW'"
data = pd.read_sql(sql, con=engine, )

data

In [ ]:
cdf = pd.read_parquet('station_config/station_data/all_data.parquet')
cdf.replace(-9999,np.nan,inplace=True)
cdf['le_pos'] = cdf['LE'].apply(lambda x: np.nan if x < 0 else x)
cdf['et_pos'] = cdf['ET'].apply(lambda x: np.nan if x < 0 else x)

utw= cdf.loc['US-UTW']#.loc[pd.to_datetime('2021-01-15'):pd.to_datetime('2022-07-14')].copy(deep=True)

utw = utw.dropna(axis=1, how='all')
#for col in utd.columns:
#    if "SSITC" in col:
#        utd.drop([col],axis=1,inplace=True)
#td.drop(['station','TIMESTAMP_END','TIMESTAMP_START'],axis=1,inplace=True)

In [ ]:
cdf

In [ ]:
import plotly.express as px
import pandas as pd


fig = px.line(utw, x=utw.index, y=['SWC_1_1_1','SWC_2_1_1'], title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

cdf = pd.read_parquet('station_config/station_data/all_data.parquet')
cdf.replace(-9999,np.nan,inplace=True)
cdf['le_pos'] = cdf['LE'].apply(lambda x: np.nan if x < 0 else x)
cdf['et_pos'] = cdf['ET'].apply(lambda x: np.nan if x < 0 else x)

utw= cdf.loc['US-UTW']#.loc[pd.to_datetime('2021-01-15'):pd.to_datetime('2022-07-14')].copy(deep=True)

utw = utw.dropna(axis=1, how='all')



def find_irr_dates(df, swc_col='SWC_1_1_1', do_plot=False, dist=20, height=35, prom=0.5):
    """
    Finds irrigation dates within a DataFrame.

    :param df: A pandas DataFrame containing the data.
    :param swc_col: String. The column name in 'df' containing the soil water content data. Should be in units of percent and not a decimal; Default is 'SWC_1_1_1'.
    :param do_plot: Boolean. Whether to plot the irrigation dates on a graph. Default is False.
    :param dist: Integer. The minimum number of time steps between peaks in 'swc_col'. Default is 20.
    :param height: Integer. The minimum height (vertical distance) of the peaks in 'swc_col'. Default is 35(%).
    :param prom: Float. The minimum prominence of the peaks in 'swc_col'. Default is 0.5.

    :return: A tuple containing the irrigation dates and the corresponding soil water content values.
    """
    df_irr_season = df[df.index.month.isin([4,5,6,7,8,9,10])]
    peaks, _ = find_peaks(df_irr_season[swc_col], distance=dist, height=height,prominence=(prom,None))
    dates_of_irr = df_irr_season.iloc[peaks].index
    swc_during_irr = df_irr_season[swc_col].iloc[peaks]
    if do_plot:
        plt.plot(df.index,df[swc_col])
        plt.plot(dates_of_irr,swc_during_irr, "x")
        plt.show()
    return dates_of_irr, swc_during_irr

find_irr_dates(utw, 'SWC_1_1_1',do_plot=True)

In [ ]:
utw.iloc[peaks].index

In [ ]:
camp = pd.read_csv("C:/Users/paulinkenbrandt/Downloads/Wellington_Flux_CSFormat.dat",skiprows=[0,2,3],index_col='TIMESTAMP',
                   parse_dates=True)
camp = camp[camp.index < pd.to_datetime('2025-01-01')]
fig = px.line(camp, x=camp.index, y='TA_1_1_1', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

camp

In [ ]:
fig = px.line(utw, x=utw.index, y=['TA_1_1_1','TA_1_2_1','TA_1_3_1'], title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0, max_iter=30, missing_values=np.nan,
                            sample_posterior=True,n_nearest_features=100,)
imp_mean.fit(utd)
#X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
new_utd = pd.DataFrame(imp_mean.transform(utd), columns=utd.columns)
utd['imp_LE'] = new_utd['LE']


In [ ]:
fig = px.line(utd, y=['LE'], title='Time Series with Range Slider and Selectors')


fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

In [ ]:


new_utd.index = utd.index
new_utd['NETRAD'].resample('1D').sum().plot()
new_utd['LE'].resample('1D').sum().plot()
new_utd['G'].resample('1D').sum().plot()
new_utd['H'].resample('1D').sum().plot()


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

dlybalance = new_utd[['LE','NETRAD','G','H']].resample('1D').sum()#.plot()
plt.scatter(dlybalance['LE']+dlybalance['H'],dlybalance['NETRAD']-dlybalance['G'])
plt.plot([1000,8000],[1000,8000])
plt.grid()

In [ ]:
pd.read_csv(r"C:\Users\paulinkenbrandt\Downloads\Dugout Ranch_Flux_AmeriFluxFormat.dat",skiprows=[0,2,3])

In [ ]:
sql = "SELECT * FROM groundwater.amfluxeddy"
df = pd.read_sql(sql, engine)
existing = df.station.unique()

site_folders = {'US-UTD':'Dugout_Ranch','US-UTB':'BSF','US-UTJ':'Bluff','US-UTW':'Wellington','US-UTE':'Escalante',
                'US-UTM':'Matheson','US-UTP':'Phrag','US-CdM':'Cedar_mesa','US-UTV':'Desert_View_Myton'}

for key, value in site_folders.items():
    if key in existing:
        pass
    else:
        print(key)
        raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/')
        raw_data = flux_network.raw_file_compile(raw_fold, value)
        if raw_data is not None:
            am_data = flux_network.Reformatter(raw_data)
    
            am_df = am_data.et_data
            am_df['station'] = key
            am_df = flux_network.remove_extra_soil_params(am_df)
    
            print(len(raw_data))
            print(len(am_df))
    
            am_df.to_sql(name = 'amfluxeddy',schema='groundwater', con=engine, if_exists='append', chunksize=2000)
            #am_df.to_csv(f'{key}_amfluxeddy.csv')

In [ ]:
sql = "SELECT * FROM groundwater.amfluxeddy"
df = pd.read_sql(sql, engine)
statdf = df.set_index(['station','datetime_start'])
for station in statdf.index.get_level_values(0).unique():
    statdf.loc[station,'ET'].plot()

In [ ]:
statdf.loc['US-UTD']